In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    %cd 'drive/My Drive/DATA_MINING_PROJECT/'
except ImportError as e:
    pass

Mounted at /content/drive/
/content/drive/My Drive/DATA_MINING_PROJECT


In [3]:
# Load the datasets
train_data = 'Data_Mining-master/train.csv'
train_df = pd.read_csv(train_data)


In [4]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [5]:
# Convert 'date' columns to datetime
train_df['date'] = pd.to_datetime(train_df['date'])

In [6]:
# Encode the 'family' categorical data
label_encoder = LabelEncoder()
train_df['family_encoded'] = label_encoder.fit_transform(train_df['family'])

In [7]:
# Aggregate data to daily sales per store per family
daily_train_df = train_df.groupby(['date','store_nbr', 'family_encoded']).agg({
    'sales': 'sum',
    'onpromotion': 'mean'
}).reset_index()


In [8]:
daily_train_df.head()

,date,store_nbr,family_encoded,sales,onpromotion
0,2013-01-01,1,0,0.0,0.0
1,2013-01-01,1,1,0.0,0.0
2,2013-01-01,1,2,0.0,0.0
3,2013-01-01,1,3,0.0,0.0
4,2013-01-01,1,4,0.0,0.0


In [9]:
len(daily_train_df)

3000888

In [26]:
# Features and target for the model
features = daily_train_df[[ 'family_encoded', 'store_nbr','onpromotion']]
target = daily_train_df['sales']

In [27]:
# Splitting the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.2, random_state=42)

In [28]:
X_train.head()

,family_encoded,store_nbr,onpromotion
2902383,0,40,0.0
1651514,29,42,0.0
1948349,29,19,0.0
2893871,2,52,1.0
2867501,32,8,6.0


In [30]:
# Initialize and train the RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42, min_samples_leaf=4, min_samples_split=2)
rf_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, min_samples_leaf=4, n_estimators=200,
                      random_state=42)

In [31]:
# Predict on the validation set
val_predictions = rf_model.predict(X_val)

# Calculate Model Score
print('The model score is: ',rf_model.score(X_val,y_val))

# Calculate MSE
val_mse = mean_squared_error(y_val, val_predictions)

# Calculate RMSE
val_rmse = np.sqrt(val_mse)

# Calculate MAE
val_mae = mean_absolute_error(y_val, val_predictions)

# Print the results
print("Validation MSE:", val_mse)
print("Validation RMSE:", val_rmse)
print("Validation MAE:", val_mae)

The model score is:  0.8076777231074117
Validation MSE: 233168.67577283634
Validation RMSE: 482.8754246934051
Validation MAE: 150.3514590293663
